<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"

In [ ]:
!pip install streamlit transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 128.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 120.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 130.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
model_name = "skt/kogpt2-base-v2"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_name,
#                                           bos_token='</s>',
#                                           eos_token='</s>',
#                                           unk_token='<unk>',
#                                           pad_token='<pad>',
#                                           mask_token='<mask>',
#                                           max_len=1024)

In [ ]:
# tokenizer.save_pretrained(f'{DATA_PATH}tokenizer')

('/content/drive/MyDrive/Final_Project/data/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Final_Project/data/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Final_Project/data/tokenizer/vocab.json',
 '/content/drive/MyDrive/Final_Project/data/tokenizer/merges.txt',
 '/content/drive/MyDrive/Final_Project/data/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/Final_Project/data/tokenizer/tokenizer.json')

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e0b902f287cd686b0832956fd91224b86bb9c1258e0f599dee2f284e644a8c0c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
!pip install -q streamlit

In [ ]:
!pip install streamlit-option-menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.2/799.2 kB 13.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import numpy as np
import torch
import re
import time
from PIL import Image
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score

bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 유사도 평가 함수
def evaluate_similarity(input_text, generated_text, alpha=0.9):
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))
    # jaccard_sim = jaccard_score(input_tokens, generated_tokens)

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

# 모델 로드
def loaded_model(DATA_PATH):
    model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}sohyun_23762_99_loss_0.1528")
    return model

# 토크나이저 로드
def loaded_tokenizer(DATA_PATH):
    tokenizer = AutoTokenizer.from_pretrained(f"{DATA_PATH}tokenizer")
    return tokenizer

# 답변 생성 함수
def generate_response(model, tokenizer, input_text, num_samples):
    text = "<q>" + input_text + "</s><a>"
    x = tokenizer.encode(text, return_tensors="pt")
    q_len = len(text) + 1

    best_generated_text = None
    best_similarity_score = -1.0
    generated_texts = []  # 답변 후보군을 담을 리스트
    for i in range(num_samples):
        result_ids = model.generate(x,
                                    max_length=1024,
                                    repetition_penalty=2.0,
                                    use_cache=True,
                                    do_sample=True,
                                    temperature=0.8,
                                    top_k=50,
                                    num_return_sequences=1
                                    )
        generated_text = tokenizer.decode(result_ids[0])
        generated_text = generated_text[q_len:-4]
        generated_text = re.sub(r'[^가-힣.]', ' ', generated_text)

        similarity_score = evaluate_similarity(text, generated_text)
        generated_texts.append((similarity_score,generated_text))
        print(generated_texts[i])

        if similarity_score > best_similarity_score:
            best_similarity_score = similarity_score
            best_generated_text = generated_text

    print("Best_similarity_score:", best_similarity_score)

    return best_generated_text

# 메인 함수 정의
def main():
    #url = "https://t1.daumcdn.net/thumb/R720x0/?fname=http://t1.daumcdn.net/brunch/service/user/6a9B/image/e-6NavAQbiquCnZ1ANdRGunwUek.png"
    #st.image(url)


    DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"

    # 사이드바(옵션)
    with st.sidebar:
        choice = option_menu("Menu", ["사장님 답글 남기기", "리뷰 분석", "기타"],
                         icons=['house', 'kanban', 'bi bi-robot'],
                         menu_icon="app-indicator", default_index=0,
                         styles={
        "container": {"padding": "4!important", "background-color": "#fafafa"},
        "icon": {"color": "black", "font-size": "25px"},
        "nav-link": {"font-size": "16px", "color": "black", "text-align": "left", "margin":"0px", "--hover-color": "#cae8e6"},
        "nav-link-selected": {"background-color": "#08c7b4"},
                                }
        )

    # 사장님 답글 페이지
    if choice == "사장님 답글 남기기":
        st.title("사장님을 위한 답변 추천 공간")
        user_input = st.text_input(label="고객 리뷰", value="")
        submit_button = st.button("Submit")

        if user_input:
            model = loaded_model(DATA_PATH)
            tokenizer = loaded_tokenizer(DATA_PATH)

            if submit_button:
                if user_input:
                    with st.spinner("답변 생성 중입니다..."):
                        reply = generate_response(model, tokenizer, user_input, num_samples = 3)
                    st.success("완료!")
                    st.write("답글을 추천해 드려요. : ")
                    st.write(reply)
                else:
                    st.warning("고객 리뷰를 확인하세요.")

    # 리뷰 분석 페이지
    elif choice == "리뷰 분석":
        st.title("사장님을 위한 리뷰 분석 공간")
        df = pd.read_csv(f"{DATA_PATH}result_fin.csv")
        label_counts = df['category'].value_counts()
        labels = label_counts.index[:5]
        sizes = label_counts.values[:5]
        plt.rc('font', family='NanumBarunGothic')

        fig, ax = plt.subplots()
        ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)

        plt.title('부정 리뷰 현황')

        st.pyplot(fig)

    elif choice == "기타":
        pass

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
    # # 사이드바에 옵션 메뉴 추가
    # st.sidebar.title("옵션")
    # num_samples = st.sidebar.number_input("답변 후보 생성 수", min_value=1, max_value=10, value=3)

In [ ]:
    # with st.sidebar:
    # choice = option_menu("Menu", ["사장님댓글 남기기", "리뷰 분석", "기타"],
    #                      icons=['house', 'kanban', 'bi bi-robot'],
    #                      menu_icon="app-indicator", default_index=0,
    #                      styles={
    #     "container": {"padding": "4!important", "background-color": "#fafafa"},
    #     "icon": {"color": "black", "font-size": "25px"},
    #     "nav-link": {"font-size": "16px", "text-align": "left", "margin":"0px", "--hover-color": "#fafafa"},
    #     "nav-link-selected": {"background-color": "#08c7b4"},
    # }
    # )

In [ ]:
# 3
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# "Password/Enpoint IP for localtunnel is:" 우측에 xx.xxx.xx.xxx 혹은 xx.xxx.xxx.xxx 형식의 숫자가 나온다.

Password/Enpoint IP for localtunnel is: 35.225.86.42


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.614s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

# "your url is:" 우측에 사이트 주소가 생성된다.

npx: installed 22 in 1.877s
your url is: https://bumpy-spies-guess.loca.lt
^C
